<a href="https://colab.research.google.com/github/Vaishno2205/disease-prediction-ai/blob/main/Disease_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving dataset.csv to dataset (1).csv


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving symptom_Description.csv to symptom_Description.csv


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving symptom_precaution.csv to symptom_precaution.csv


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving Symptom-severity.csv to Symptom-severity.csv


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


In [ ]:
df = pd.read_csv("dataset.csv")
df = pd.read_csv("symptom_Description.csv")
df = pd.read_csv("symptom_precaution.csv")
df = pd.read_csv("Symptom-severity.csv")
df.head()
df.head()
df.head()
df.head()


,Symptom,weight
0,itching,1
1,skin_rash,3
2,nodal_skin_eruptions,4
3,continuous_sneezing,4
4,shivering,5


In [ ]:
print(df.columns)

Index(['Symptom', 'weight'], dtype='object')


In [ ]:
print(df.columns.tolist())

['Symptom', 'weight']


In [ ]:
['Symptom_1', 'Symptom_2', ..., 'prognosis']

['Symptom_1', 'Symptom_2', Ellipsis, 'prognosis']

In [ ]:
y = df['Symptom']
X = df.drop('Symptom', axis=1)


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving dataset.csv to dataset (2).csv


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings("ignore")

# 1️⃣ Load your dataset
dataset = pd.read_csv("/content/dataset.csv")

# Print columns of the loaded dataset to verify
print("Columns of dataset:", dataset.columns)

# 2️⃣ Preprocess the main dataset
# Replace NaN with a placeholder string, or handle them during unique symptom extraction
# Using a placeholder string like 'Unknown'
dataset = dataset.fillna('Unknown')

# Separate features (X) and target (y) before encoding
X = dataset.drop('Disease', axis=1)
y = dataset['Disease']

# Convert symptom text into binary (One-Hot encoding)
# Get all unique symptoms from the symptom columns, ensuring they are strings
symptom_cols = [col for col in dataset.columns if 'Symptom' in col]
all_symptoms = sorted(set(str(symptom) for col in symptom_cols for symptom in dataset[col].dropna().unique()))
# Remove the 'Unknown' placeholder if it exists
if 'Unknown' in all_symptoms:
    all_symptoms.remove('Unknown')


# Initialize a new DataFrame for binary encoding with columns for all unique symptoms
symptom_data = pd.DataFrame(0, index=dataset.index, columns=all_symptoms)

# Mark 1 where a symptom is present in a row
for idx, row in dataset.iterrows():
    for col in symptom_cols:
        symptom = str(row[col]) # Ensure symptom is a string
        if symptom != 'Unknown': # Exclude the placeholder
            if symptom in symptom_data.columns:
                symptom_data.loc[idx, symptom] = 1


# 3️⃣ Encode target column (Disease)
le = LabelEncoder()
y_encoded = le.fit_transform(y)


# 4️⃣ Split dataset
X_train, X_test, y_train, y_test = train_test_split(symptom_data, y_encoded, test_size=0.2, random_state=42)

# 5️⃣ Train the model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# 6️⃣ Evaluate
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=le.classes_))

Columns of dataset: Index(['Disease', 'Symptom_1', 'Symptom_2', 'Symptom_3', 'Symptom_4',
       'Symptom_5', 'Symptom_6', 'Symptom_7', 'Symptom_8', 'Symptom_9',
       'Symptom_10', 'Symptom_11', 'Symptom_12', 'Symptom_13', 'Symptom_14',
       'Symptom_15', 'Symptom_16', 'Symptom_17'],
      dtype='object')
Accuracy: 1.0
                                         precision    recall  f1-score   support

(vertigo) Paroymsal  Positional Vertigo       1.00      1.00      1.00        18
                                   AIDS       1.00      1.00      1.00        30
                                   Acne       1.00      1.00      1.00        24
                    Alcoholic hepatitis       1.00      1.00      1.00        25
                                Allergy       1.00      1.00      1.00        24
                              Arthritis       1.00      1.00      1.00        23
                       Bronchial Asthma       1.00      1.00      1.00        33
                   Cervica

In [ ]:
# Load disease descriptions
desc_df = pd.read_csv("/content/symptom_Description.csv")

# If you have precautions file (if not, skip this)
prec_df = pd.read_csv("/content/symptom_precaution.csv") # Changed to symptom_precaution.csv

def predict_disease(user_symptoms):
    # Create a DataFrame with the same columns as the training data (symptom_data from jFtBc6Rk5xiQ), initialized to 0
    # Assuming symptom_data and le (LabelEncoder) are available from the previous cell (jFtBc6Rk5xiQ)
    input_data = pd.DataFrame(0, index=[0], columns=symptom_data.columns)

    for symptom in user_symptoms:
        # Convert symptom to lowercase and replace spaces with underscores for column matching
        symptom_formatted = symptom.lower().replace(' ', '_')
        # Find the corresponding column in the one-hot encoded data and set the value to 1
        if symptom_formatted in input_data.columns:
            input_data[symptom_formatted] = 1
        else:
            # Handle cases where the symptom is not found in the training data
            print(f"Warning: Symptom '{symptom}' (formatted as '{symptom_formatted}') not found in training data.")


    # Predict
    prediction = model.predict(input_data)[0]
    disease_name = le.inverse_transform([prediction])[0]

    # Get description
    description = desc_df[desc_df['Disease'] == disease_name]['Description'].values
    description = description[0] if len(description) > 0 else "No description available."

    # Get precautions (if available)
    # Make sure to drop the 'Disease' column before converting to list
    precautions = prec_df[prec_df['Disease'] == disease_name].drop(columns=['Disease'], errors='ignore').values
    precautions = precautions[0].tolist() if len(precautions) > 0 else ["No precautions found."]

    return disease_name, description, precautions

# Example usage
# Ensure symptom names match the format in your dataset for best results
symptoms_list = ['itching', 'skin_rash', 'joint_pain']
disease, desc, precs = predict_disease(symptoms_list)

print("\nPredicted Disease:", disease)
print("\nDescription:", desc)
print("\nPrecautions:")
for i, p in enumerate(precs, 1):
    print(f"{i}. {p}")


Predicted Disease: Fungal infection

Description: In humans, fungal infections occur when an invading fungus takes over an area of the body and is too much for the immune system to handle. Fungi can live in the air, soil, water, and plants. There are also some fungi that live naturally in the human body. Like many microbes, there are helpful fungi and harmful fungi.

Precautions:
1. bath twice
2. use detol or neem in bathing water
3. keep infected area dry
4. use clean cloths


In [ ]:
import os
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings("ignore")

base_paths = ["/mnt/data", "./", "/content/"]
def try_read(fname):
    for p in base_paths:
        fp = os.path.join(p, fname)
        if os.path.exists(fp):
            return pd.read_csv(fp)
    raise FileNotFoundError(f"{fname} not found in {base_paths}")

print("Loading datasets...")
dataset = try_read("dataset.csv")
print(" Loaded dataset.csv:", dataset.shape)
try:
    desc_df = try_read("symptom_Description.csv")
    print(" Loaded symptom_Description.csv:", desc_df.shape)
except FileNotFoundError:
    desc_df = pd.DataFrame(columns=["Disease","Description"])
    print(" symptom_Description.csv not found -> continuing without descriptions.")
try:
    prec_df = try_read("symptom_precaution.csv")
    print(" Loaded symptom_precaution.csv:", prec_df.shape)
except FileNotFoundError:
    prec_df = pd.DataFrame()
    print(" symptom_precaution.csv not found -> continuing without precautions.")
try:
    sev_df = try_read("Symptom-severity.csv")
    print(" Loaded Symptom-severity.csv:", sev_df.shape)
except FileNotFoundError:
    sev_df = pd.DataFrame()
    print(" Symptom-severity.csv not found -> continuing without severity data.")

possible_targets = ['prognosis','Prognosis','disease','Disease','Diagnosis','diagnosis','target']
target_col = None
for c in dataset.columns:
    if c in possible_targets:
        target_col = c
        break
if target_col is None:
    target_col = dataset.columns[-1]
print("Using target column:", target_col)

feature_cols = [c for c in dataset.columns if c != target_col]
numeric_count = sum(pd.api.types.is_numeric_dtype(dataset[c]) for c in feature_cols)
wide_mode = False
if numeric_count >= max(1, int(0.6 * len(feature_cols))):
    is_binary = all(set(dataset[c].dropna().unique()).issubset({0,1}) for c in feature_cols if pd.api.types.is_numeric_dtype(dataset[c]))
    if is_binary:
        wide_mode = True

print("wide_mode:", wide_mode)

if wide_mode:
    X = dataset[feature_cols].fillna(0).astype(int)
else:

    print("Converting long-format data (symptom names -> binary features)...")
    symptom_cols = feature_cols
    all_symptoms = set()
    for col in symptom_cols:
        all_symptoms.update(dataset[col].dropna().astype(str).unique())
    all_symptoms = sorted([s for s in all_symptoms if str(s).strip() != '' and str(s).lower() != str(dataset[target_col].name).lower()])
    print(f"Found {len(all_symptoms)} distinct symptoms.")

    X = pd.DataFrame(0, index=dataset.index, columns=all_symptoms)
    for idx, row in dataset.iterrows():
        for col in symptom_cols:
            val = row[col]
            if pd.isna(val): continue
            sval = str(val).strip()
            if sval == '': continue
            if sval in X.columns:
                X.at[idx, sval] = 1

y_raw = dataset[target_col].astype(str).str.strip()
le = LabelEncoder()
y = le.fit_transform(y_raw)
disease_classes = le.classes_
print("Number of disease classes:", len(disease_classes))

symptom_list = list(X.columns)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print("Train shape:", X_train.shape, "Test shape:", X_test.shape)

model = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
print("Training model...")
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Test accuracy: {acc:.4f}")
print("\nClassification report:")
print(classification_report(y_test, y_pred, target_names=disease_classes))

def predict_from_symptoms(symptom_input_list, top_k=3):
    """
    symptom_input_list: list of symptom names (strings)
    top_k: how many top diseases to return (by probability)
    Returns: list of tuples (disease_name, probability)
    """
    vec = np.zeros(len(symptom_list), dtype=int)
    for s in symptom_input_list:
        s = s.strip()
        if s in symptom_list:
            vec[symptom_list.index(s)] = 1
    probs = model.predict_proba([vec])[0]  # shape = (n_classes,)
    top_idx = np.argsort(probs)[-top_k:][::-1]
    results = [(disease_classes[i], float(probs[i])) for i in top_idx]
    return results

def get_description(disease_name):
    if 'Disease' in desc_df.columns:
        row = desc_df[desc_df['Disease'].astype(str).str.strip() == disease_name]
        if not row.empty:
            if 'Description' in row.columns:
                return str(row['Description'].values[0])
            # fallback try other columns
            for c in row.columns:
                if c.lower().startswith('desc'):
                    return str(row[c].values[0])
    return "No description available."

def get_precautions(disease_name):
    if 'Disease' in prec_df.columns:
        row = prec_df[prec_df['Disease'].astype(str).str.strip() == disease_name]
        if not row.empty:
            steps = row.drop(columns=['Disease']).values.flatten().tolist()
            steps = [str(s) for s in steps if str(s).strip().lower() not in ['nan','none','']]
            return steps
    return []

print("\nExample prediction (change symptoms below):")
example_symptoms = symptom_list[:6]
print("Example symptoms sample:", example_symptoms)
res = predict_from_symptoms(example_symptoms, top_k=3)
for name, p in res:
    print(f" - {name}: {p:.3f}")
    print("   Description:", get_description(name)[:140])
    precs = get_precautions(name)
    if precs:
        print("   Precautions:", precs[:3])
    else:
        print("   Precautions: none found")

os.makedirs("/mnt/data/model_artifacts", exist_ok=True)
joblib.dump(model, "/mnt/data/model_artifacts/disease_model_rf.joblib")
joblib.dump(le, "/mnt/data/model_artifacts/label_encoder.joblib")
joblib.dump(symptom_list, "/mnt/data/model_artifacts/symptom_list.joblib")
print("\nModel and artifacts saved to /mnt/data/model_artifacts/")


Loading datasets...
 Loaded dataset.csv: (4920, 18)
 Loaded symptom_Description.csv: (41, 2)
 Loaded symptom_precaution.csv: (41, 5)
 Loaded Symptom-severity.csv: (133, 2)
Using target column: Disease
wide_mode: False
Converting long-format data (symptom names -> binary features)...
Found 131 distinct symptoms.
Number of disease classes: 41
Train shape: (3936, 131) Test shape: (984, 131)
Training model...
Test accuracy: 0.0467

Classification report:
                                         precision    recall  f1-score   support

(vertigo) Paroymsal  Positional Vertigo       0.00      0.00      0.00        24
                                   AIDS       0.00      0.00      0.00        24
                                   Acne       0.00      0.00      0.00        24
                    Alcoholic hepatitis       0.00      0.00      0.00        24
                                Allergy       0.00      0.00      0.00        24
                              Arthritis       0.00      0.

In [ ]:
import joblib
model = joblib.load("/mnt/data/model_artifacts/disease_model_rf.joblib")
le = joblib.load("/mnt/data/model_artifacts/label_encoder.joblib")
symptom_list = joblib.load("/mnt/data/model_artifacts/symptom_list.joblib")

user_input = input("Enter comma-separated symptoms: ")
user_symptoms = [s.strip() for s in user_input.split(",") if s.strip()]
results = predict_from_symptoms(user_symptoms, top_k=3)
print("Top predictions:")
for disease, prob in results:
    print(f"{disease} ({prob:.3f})")
    print("Description:", get_description(disease))
    print("Precautions:", get_precautions(disease))
    print("---")


Enter comma-separated symptoms: headache, vomiting, nauseus
Top predictions:
Migraine (0.029)
Description: A migraine can cause severe throbbing pain or a pulsing sensation, usually on one side of the head. It's often accompanied by nausea, vomiting, and extreme sensitivity to light and sound. Migraine attacks can last for hours to days, and the pain can be so severe that it interferes with your daily activities.
Precautions: ['meditation', 'reduce stress', 'use poloroid glasses in sun', 'consult doctor']
---
Acne (0.029)
Description: Acne vulgaris is the formation of comedones, papules, pustules, nodules, and/or cysts as a result of obstruction and inflammation of pilosebaceous units (hair follicles and their accompanying sebaceous gland). Acne develops on the face and upper trunk. It most often affects adolescents.
Precautions: ['bath twice', 'avoid fatty spicy food', 'drink plenty of water', 'avoid too many products']
---
AIDS (0.029)
Description: Acquired immunodeficiency syndrome 

In [ ]:
# --- Streamlit app (save this block to app.py) ---
"""
import streamlit as st
import joblib

# Load artifacts
model = joblib.load("/mnt/data/model_artifacts/disease_model_rf.joblib")
le = joblib.load("/mnt/data/model_artifacts/label_encoder.joblib")
symptom_list = joblib.load("/mnt/data/model_artifacts/symptom_list.joblib")
# helper functions: predict_from_symptoms, get_description, get_precautions (copy from notebook above)

st.title("Disease Prediction from Symptoms")
st.write("Select symptoms you are experiencing:")

selected = st.multiselect("Symptoms", symptom_list)

if st.button("Predict"):
    if not selected:
        st.warning("Please select at least one symptom.")
    else:
        res = predict_from_symptoms(selected, top_k=5)
        st.subheader("Top Predictions")
        for name, prob in res:
            st.markdown(f"**{name}** — {prob:.2%}")
            st.write(get_description(name))
            precs = get_precautions(name)
            if precs:
                st.write("Precautions:")
                for p in precs:
                    st.write("-", p)
"""


'\nimport streamlit as st\nimport joblib\n\n# Load artifacts\nmodel = joblib.load("/mnt/data/model_artifacts/disease_model_rf.joblib")\nle = joblib.load("/mnt/data/model_artifacts/label_encoder.joblib")\nsymptom_list = joblib.load("/mnt/data/model_artifacts/symptom_list.joblib")\n# helper functions: predict_from_symptoms, get_description, get_precautions (copy from notebook above)\n\nst.title("Disease Prediction from Symptoms")\nst.write("Select symptoms you are experiencing:")\n\nselected = st.multiselect("Symptoms", symptom_list)\n\nif st.button("Predict"):\n    if not selected:\n        st.warning("Please select at least one symptom.")\n    else:\n        res = predict_from_symptoms(selected, top_k=5)\n        st.subheader("Top Predictions")\n        for name, prob in res:\n            st.markdown(f"**{name}** — {prob:.2%}")\n            st.write(get_description(name))\n            precs = get_precautions(name)\n            if precs:\n                st.write("Precautions:")\n      

In [ ]:
st \n import joblib\n\n

SyntaxError: unexpected character after line continuation character (ipython-input-4248045869.py, line 1)

In [ ]:
%%writefile app.py

import streamlit as st
import joblib
import pandas as pd
import numpy as np


# Load artifacts (Assuming these were saved in a previous step to /mnt/data/model_artifacts/)
try:
    model = joblib.load("/mnt/data/model_artifacts/disease_model_rf.joblib")
    le = joblib.load("/mnt/data/model_artifacts/label_encoder.joblib")
    symptom_list = joblib.load("/mnt/data/model_artifacts/symptom_list.joblib")
except FileNotFoundError:
    st.error("Model artifacts not found. Please run the model training cell first to save the artifacts.")
    st.stop() # Stop execution if artifacts are not found

# Load description and precaution data (Assuming these CSVs are in /content/)
try:
    desc_df = pd.read_csv("/content/symptom_Description.csv")
except FileNotFoundError:
    desc_df = pd.DataFrame(columns=["Disease","Description"])
    st.warning("symptom_Description.csv not found. Descriptions will not be available.")

try:
    prec_df = pd.read_csv("/content/symptom_precaution.csv")
except FileNotFoundError:
    prec_df = pd.DataFrame()
    st.warning("symptom_precaution.csv not found. Precautions will not be available.")


# Helper function to predict disease from symptoms (adapted from notebook code)
def predict_from_symptoms(symptom_input_list, top_k=3):
    """
    symptom_input_list: list of symptom names (strings)
    top_k: how many top diseases to return (by probability)
    Returns: list of tuples (disease_name, probability)
    """
    # Ensure symptom_list is available (loaded from joblib)
    if 'symptom_list' not in globals():
         st.error("symptom_list not loaded. Cannot predict.")
         return []

    vec = np.zeros(len(symptom_list), dtype=int)
    for s in symptom_input_list:
        s = s.strip()
        # Symptom list loaded from joblib should contain the correctly formatted symptom names
        if s in symptom_list:
            vec[symptom_list.index(s)] = 1
        else:
            st.warning(f"Symptom '{s}' not found in known symptoms.")

    # Ensure model is available (loaded from joblib)
    if 'model' not in globals():
        st.error("Model not loaded. Cannot predict.")
        return []

    probs = model.predict_proba([vec])[0]  # shape = (n_classes,)
    top_idx = np.argsort(probs)[-top_k:][::-1]

    # Ensure le (LabelEncoder) is available
    if 'le' not in globals():
        st.error("Label encoder not loaded. Cannot predict.")
        return []

    results = [(le.classes_[i], float(probs[i])) for i in top_idx]
    return results

# Helper function to get description (adapted from notebook code)
def get_description(disease_name):
    if 'desc_df' in globals() and 'Disease' in desc_df.columns:
        row = desc_df[desc_df['Disease'].astype(str).str.strip() == disease_name]
        if not row.empty:
            if 'Description' in row.columns:
                return str(row['Description'].values[0])
            # fallback try other columns
            for c in row.columns:
                if c.lower().startswith('desc'):
                    return str(row[c].values[0])
    return "No description available."

# Helper function to get precautions (adapted from notebook code)
def get_precautions(disease_name):
    if 'prec_df' in globals() and 'Disease' in prec_df.columns:
        row = prec_df[prec_df['Disease'].astype(str).str.strip() == disease_name]
        if not row.empty:
            # Get all columns except 'Disease' and filter out empty/NaN values
            steps = row.drop(columns=['Disease'], errors='ignore').values.flatten().tolist()
            steps = [str(s).strip() for s in steps if pd.notna(s) and str(s).strip().lower() not in ['nan','none','']]
            return steps
    return []


# --- Streamlit App Layout ---
st.title("Disease Prediction from Symptoms")
st.write("Select symptoms you are experiencing:")

# Use the loaded symptom_list for the multiselect options
selected = st.multiselect("Symptoms", symptom_list)

if st.button("Predict"):
    if not selected:
        st.warning("Please select at least one symptom.")
    else:
        st.info("Predicting...")
        res = predict_from_symptoms(selected, top_k=5) # Predict top 5 for the app
        st.subheader("Top Predictions")
        if res:
            for name, prob in res:
                st.markdown(f"**{name}** — {prob:.2%}")
                description = get_description(name)
                if description:
                    st.write(description)
                precs = get_precautions(name)
                if precs:
                    st.write("Precautions:")
                    for p in precs:
                        st.write("-", p)
                st.markdown("---") # Separator for predictions
        else:
            st.write("Could not make a prediction based on the selected symptoms.")

Writing app.py


In [ ]:
# --- Flask API (save as app_flask.py) ---
"""
from flask import Flask, request, jsonify
import joblib
model = joblib.load("/mnt/data/model_artifacts/disease_model_rf.joblib")
le = joblib.load("/mnt/data/model_artifacts/label_encoder.joblib")
symptom_list = joblib.load("/mnt/data/model_artifacts/symptom_list.joblib")
# Use predict_from_symptoms, get_description, get_precautions from notebook

app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    data = request.json
    symptoms = data.get('symptoms', [])
    top_k = int(data.get('top_k', 3))
    res = predict_from_symptoms(symptoms, top_k=top_k)
    out = []
    for name, prob in res:
        out.append({
            "disease": name,
            "probability": prob,
            "description": get_description(name),
            "precautions": get_precautions(name)
        })
    return jsonify({"predictions": out})

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)
"""


'\nfrom flask import Flask, request, jsonify\nimport joblib\nmodel = joblib.load("/mnt/data/model_artifacts/disease_model_rf.joblib")\nle = joblib.load("/mnt/data/model_artifacts/label_encoder.joblib")\nsymptom_list = joblib.load("/mnt/data/model_artifacts/symptom_list.joblib")\n# Use predict_from_symptoms, get_description, get_precautions from notebook\n\napp = Flask(__name__)\n\n@app.route(\'/predict\', methods=[\'POST\'])\ndef predict():\n    data = request.json\n    symptoms = data.get(\'symptoms\', [])\n    top_k = int(data.get(\'top_k\', 3))\n    res = predict_from_symptoms(symptoms, top_k=top_k)\n    out = []\n    for name, prob in res:\n        out.append({\n            "disease": name,\n            "probability": prob,\n            "description": get_description(name),\n            "precautions": get_precautions(name)\n        })\n    return jsonify({"predictions": out})\n\nif __name__ == "__main__":\n    app.run(host=\'0.0.0.0\', port=5000)\n'

In [ ]:
import pandas as pd
import joblib
import numpy as np

# Load datasets
desc_df = pd.read_csv("symptom_Description.csv")
prec_df = pd.read_csv("symptom_precaution.csv")

# Function to get disease description
def get_description(disease):
    try:
        return desc_df.loc[desc_df['Disease'] == disease, 'Description'].values[0]
    except:
        return "No description available."

# Function to get precautions
def get_precautions(disease):
    try:
        row = prec_df.loc[prec_df['Disease'] == disease].iloc[0]
        return [row[f'Precaution_{i}'] for i in range(1, 5)]
    except:
        return []

# Function to predict from symptoms
def predict_from_symptoms(symptoms, model, le, symptom_list, top_k=3):
    input_data = [0] * len(symptom_list)
    for s in symptoms:
        if s in symptom_list:
            input_data[symptom_list.index(s)] = 1

    pred_proba = model.predict_proba([input_data])[0]
    top_indices = np.argsort(pred_proba)[::-1][:top_k]
    diseases = le.inverse_transform(top_indices)
    return list(zip(diseases, pred_proba[top_indices]))


In [ ]:
%%writefile app_flask.py

from flask import Flask, request, jsonify
import joblib
import pandas as pd
import numpy as np
import os

# Load models and encoders
# Assuming artifacts are saved in /mnt/data/model_artifacts/
try:
    model = joblib.load("/mnt/data/model_artifacts/disease_model_rf.joblib")
    le = joblib.load("/mnt/data/model_artifacts/label_encoder.joblib")
    symptom_list = joblib.load("/mnt/data/model_artifacts/symptom_list.joblib")
except FileNotFoundError:
    print("Error: Model artifacts not found. Please run the model training cell first to save the artifacts.")
    # Exit or handle error appropriately for a Flask app, maybe return an error response

# Load datasets for descriptions and precautions
# Assuming these CSVs are in /content/
try:
    desc_df = pd.read_csv("/content/symptom_Description.csv")
except FileNotFoundError:
    desc_df = pd.DataFrame(columns=["Disease","Description"])
    print("Warning: symptom_Description.csv not found. Descriptions will not be available.")

try:
    prec_df = pd.read_csv("/content/symptom_precaution.csv")
except FileNotFoundError:
    prec_df = pd.DataFrame()
    print("Warning: symptom_precaution.csv not found. Precautions will not be available.")


# Helper function to predict disease from symptoms (adapted from notebook code)
def predict_from_symptoms(symptom_input_list, model, le, symptom_list, top_k=3):
    """
    symptom_input_list: list of symptom names (strings)
    top_k: how many top diseases to return (by probability)
    Returns: list of tuples (disease_name, probability)
    """
    if model is None or le is None or symptom_list is None:
        print("Model artifacts not loaded. Cannot predict.")
        return []

    vec = np.zeros(len(symptom_list), dtype=int)
    for s in symptom_input_list:
        s = s.strip()
        if s in symptom_list:
            vec[symptom_list.index(s)] = 1
        else:
            print(f"Warning: Symptom '{s}' not found in known symptoms.")

    try:
        probs = model.predict_proba([vec])[0]  # shape = (n_classes,)
        top_idx = np.argsort(probs)[-top_k:][::-1]
        results = [(le.classes_[i], float(probs[i])) for i in top_idx]
        return results
    except Exception as e:
        print(f"Error during prediction: {e}")
        return []


# Helper function to get description (adapted from notebook code)
def get_description(disease_name):
    if 'desc_df' in globals() and 'Disease' in desc_df.columns:
        row = desc_df[desc_df['Disease'].astype(str).str.strip() == disease_name]
        if not row.empty:
            if 'Description' in row.columns:
                return str(row['Description'].values[0])
            # fallback try other columns
            for c in row.columns:
                if c.lower().startswith('desc'):
                    return str(row[c].values[0])
    return "No description available."

# Helper function to get precautions (adapted from notebook code)
def get_precautions(disease_name):
    if 'prec_df' in globals() and 'Disease' in prec_df.columns:
        row = prec_df[prec_df['Disease'].astype(str).str.strip() == disease_name]
        if not row.empty:
            # Get all columns except 'Disease' and filter out empty/NaN values
            steps = row.drop(columns=['Disease'], errors='ignore').values.flatten().tolist()
            steps = [str(s).strip() for s in steps if pd.notna(s) and str(s).strip().lower() not in ['nan','none','']]
            return steps
    return []


app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    data = request.json
    symptoms = data.get('symptoms', [])
    top_k = int(data.get('top_k', 3))

    # Pass model, le, and symptom_list to the helper function
    results = predict_from_symptoms(symptoms, model, le, symptom_list, top_k)

    output = []
    if results:
        for disease, prob in results:
            output.append({
                "disease": disease,
                "probability": float(prob),
                "description": get_description(disease),
                "precautions": get_precautions(disease)
            })
    else:
        return jsonify({"error": "Could not make a prediction."}), 500 # Return an error if prediction failed

    return jsonify({"predictions": output})

if __name__ == "__main__":
    # Use a different port than 5000 if needed, e.g., 8000
    app.run(host='0.0.0.0', port=5000, debug=True)

Writing app_flask.py


In [ ]:
pip install flask joblib pandas numpy
python app.py


SyntaxError: invalid syntax (ipython-input-419823011.py, line 1)

After running the cell above, a file named `app.py` will be created in your Colab environment. To run the Streamlit app, execute the following command in a new code cell:

In [ ]:
pip install flask joblib pandas numpy

After installing the libraries, run the Streamlit app with the following command in a new code cell:

In [ ]:
!streamlit run app.py

/bin/bash: line 1: streamlit: command not found


In [ ]:
!curl -X POST http://127.0.0.1:5000/predict \
-H "Content-Type: application/json" \
-d '{"symptoms": ["itching", "skin_rash"], "top_k": 3}'

curl: (7) Failed to connect to 127.0.0.1 port 5000 after 0 ms: Connection refused


Alternatively, if you want to run the Flask application (`app_flask.py`), use the following command in a new code cell: